# Essential SQL for Spatial Analysis

## Introduction

## Learning Objectives

## Sample Datasets

## Setting Up the Environment

## Connecting To DuckDB

## Install Extensions

## Reading CSV Files from URLs

## Creating Tables for Better Performance

## The SQL SELECT Statement

### Limiting Results with LIMIT

### Selecting Specific Columns

### Finding Unique Values with DISTINCT

### Basic Aggregate Functions

#### Counting Features

#### Counting Unique Categories

#### Finding Extremes

#### Calculating Totals

#### Computing Averages

### Sorting Results with ORDER BY

#### Basic Sorting

#### Multi-Level Sorting

#### Spatial Sorting Applications

### Calculated Columns and Expressions

#### Basic Calculations

#### Geographic Calculations

## Filtering Data with the WHERE Clause

## Pattern Matching with LIKE

## The IN Operator

## The BETWEEN Operator

## Combining Data with SQL Joins

### SQL Inner Join

### SQL Left Join

### SQL Right Join

### SQL Full Join

### SQL Union

## Query Plans and Performance

## Aggregating Data for Summary Statistics

### Understanding GROUP BY

### Filtering Aggregated Results with HAVING

## Conditional Statements

## Saving Results

## Advanced SQL Features for Spatial Analysis

### Window Functions

#### Ranking Cities by Population

#### Moving Averages and Spatial Trends

### Common Table Expressions (CTEs)

#### Analyzing Urban Hierarchies

### Subqueries for Spatial Comparisons

#### Finding Cities Above Country Average

### String Functions for Spatial Data

#### Cleaning and Standardizing Place Names

### Date and Time Functions

## SQL Comments and Documentation

### Single-Line Comments

### Multi-Line Comments

## Key Takeaways

## Exercises

### Exercise 1: Loading Remote Data and Creating Tables

### Exercise 2: Basic Column Selection

### Exercise 3: Filtering Rows with WHERE

### Exercise 4: Sorting Results with ORDER BY

### Exercise 5: Finding Unique Values with DISTINCT

### Exercise 6: Counting and Grouping Data

### Exercise 7: Sorting Aggregated Results

### Exercise 8: Joining Related Tables

### Exercise 9: Pattern Matching with LIKE

### Exercise 10: Multiple Conditions with AND